In [15]:
'''
1. split pandas dataframe on \0000 value -
2. iter over each line - 
3. check if attribute appears from second position in first part of line - 
4. if no drop this line - 
4. store this df in a separate dataframe
6. if yes substitute attribute with value from second part of line
7. tokenize each line 
8. create a json file with df from step 4 with sentence and tokens


1. 
'''
# TODO \tSOTEK-JSWX-XHZ-72v20ah2000w Electric bicycle                     nan,SOTEK,72v   shouldnt be in the dataset
# TODO deal with null labels

'\n1. split pandas dataframe on \x000 value -\n2. iter over each line - \n3. check if attribute appears from second position in first part of line - \n4. if no drop this line - \n4. store this df in a separate dataframe\n6. if yes substitute attribute with value from second part of line\n7. tokenize each line \n8. create a json file with df from step 4 with sentence and tokens\n\n\n1. \n'

In [16]:
import pandas as pd 
import nltk
from IPython.display import display
#pd.options.display.max_colwidth = 100
#nltk.download('all')

In [17]:
# load data from data/paper folder
data = pd.read_csv('data/ACL19.txt', sep='\u0001', header=None)
print(data.head()) 

                                                   0           1    2
0  APG STO0045  Camping Stove Portable Cooking Eq...        Fuel  Gas
1  APG STO0045  Camping Stove Portable Cooking Eq...  Brand Name  APG
2  APG STO0045  Camping Stove Portable Cooking Eq...       Model  NaN
3  APG STO0045  Camping Stove Portable Cooking Eq...  Disposable  NaN
4  80LUX Mini Portable Camping Lantern Gas Light ...    Material  NaN


In [18]:
#get all values from colum 1 so that they uniquely appear in a dictionary with token
attributes = data[1].unique()

print(attributes)
with open("test.txt", "w") as f:
    for i in attributes:
        f.write(i + "\n")
    

['Fuel' 'Brand Name' 'Model' ... 'Fitness' 'Applicable Parts' 'Anti-skid']


In [19]:
#groupby same first column
data = data.groupby(0).apply(lambda x: pd.Series({"1": ",".join(x[1].astype(str)), "2": ",".join(x[2].astype(str))}))
data.head(3)

,1,2
0,,
\t2018 New Men&#39;s Basketball Shoes High Top Breathable Sneakers Ankel Boots Athletic Tenis Jordan Basketball Shoes big size shoes,"Athletic Shoe Type,Shoe Width,Feature,Gender","Basketball Shoes,nan,nan,Men"
\tHUISON S4 Professional Carbon Fiber Zebra Tec wood Table Tennis paddle/ Table Tennis Blade/ table tennis bat,"Model Number,Size","S4,nan"
\tJordan 9 XII Men Basketball Shoes wool the master University Blue gym red GS Barons Flu Game Athletic Outdoor Sport Sneakers..,Athletic Shoe Type,Basketball Shoes


In [20]:
'''
is column 1 now a string only? i can get all the unique values before i groupby everything. 

1. Get all unique values from column one
2. Create a set like tag_set = {
    "PERSON": 1,
    "ORGANIZATION": 2,
    "LOCATION": 3,
    ...
}
3. split values from column 1&2 at the comma and make them to lists
1. iter over each line 
2. if only nan types in colum 2 than drop this line
x. iter over list of colum 2 and check for each item if it appears in colum 0. if yes than replace the appearance in colum 0 with the item from colum 1 which appears on the same position like the item in colum 2
    additionally check if the replacement is contained in a another string if yes than seperate the item inside of the string with spaces 
4. i need to convert each word to tokens now but first check all the attributes look like   

Problem with Soccer gender and Soccer gender: because if I have a string with Soccer gender: and i check if Soccer gender appears than it labels it as Soccer gender but its normally Soccer gender:. In this example its not wild because its anyway the same label but if you have some like gender soccer and gender than its a problem because borh are completely wrong labels.
To avoid the gender / gender soccer proplem I could say - check first the longest strings and if they do not appear in the string go to the smaller string. soccer gender soccer - would check firt with gender soccer and than directly convert gender soccer into. 

People from the paper implemented it different. They did the following:
- removing all rows with NULL 
i still have critics over the current process because 
what about I write in my introduction that I deal with two cases. In one case i have given a dataset and make this intentionally weak and perform various models on it and than I take a dataset which is given

Than I can write my introduction like "Die Arbeit behandelt zwei Datensets. Ein Datenset bei dem NER"

In the end I want to show ways on how to do NER with weakly labelled data. My idea would be the following:

- [ ] taking the bio dataset and doing basic NER on it

I need certain steps which I reproduce for both datasets. for example ner, ner with weakly labelled data, seq2seq, 
'''

'\nis column 1 now a string only? i can get all the unique values before i groupby everything. \n\n1. Get all unique values from column one\n2. Create a set like tag_set = {\n    "PERSON": 1,\n    "ORGANIZATION": 2,\n    "LOCATION": 3,\n    ...\n}\n3. split values from column 1&2 at the comma and make them to lists\n1. iter over each line \n2. if only nan types in colum 2 than drop this line\nx. iter over list of colum 2 and check for each item if it appears in colum 0. if yes than replace the appearance in colum 0 with the item from colum 1 which appears on the same position like the item in colum 2\n    additionally check if the replacement is contained in a another string if yes than seperate the item inside of the string with spaces \n4. i need to convert each word to tokens now but first check all the attributes look like   \n\nProblem with Soccer gender and Soccer gender: because if I have a string with Soccer gender: and i check if Soccer gender appears than it labels it as Socc

In [21]:
def process_row(row):
    if str(row[2]) not in str(row[0]):
        return False
    else:
        if str(row[2]) in str(row[0]):
            row = row[0].replace(str(row[2]), str(row[1]))
            return row 
        
# now i need to apply this method to each line and if the method returns False than drop this line from the df else replace row with returnt row               
data_new = data.apply(process_row, axis=1)
#result from above should be a df where some rows are == False

#remove rows where value == False
data_new = data_new[data_new != False]

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
#print first element in first line of df
print(data_new.head())

0    APG STO0045  Camping Stove Portable Cooking Eq...
1    Brand Name STO0045  Camping Stove Portable Coo...
5    Model Card furnace Windproof Portable Stoves G...
6    BDZ-160-A Card furnace Windproof Portable Stov...
7    BDZ-160-A Card furnace Windproof Portable Stov...
dtype: object


In [ ]:
#tokenize dataframe
#nltk.download('all')
#what i want basically is to take all my words and than to turn each word into a number
#than i can iterate over the df and turn a word which is not in the attributes list into a zero
#if a word is in the list than i can substitute this word with 


#is there no method for turning each word from a dataset into a number were the same words getting the same number?
#Which method is used for labelling a dataset for named entity recognition?
#
data_new = data_new.apply(nltk.word_tokenize)

#apply bag of words
data_new = data_new.apply(nltk.bag_of_words)



AttributeError: module 'nltk' has no attribute 'bag_of_words'

In [ ]:
print(data_new.head())
#turn into number tokens


0    [(APG, NNP), (STO0045, NNP), (Camping, NNP), (...
1    [(Brand, NNP), (Name, NNP), (STO0045, NNP), (C...
5    [(Model, NNP), (Card, NNP), (furnace, NN), (Wi...
6    [(BDZ-160-A, NNP), (Card, NNP), (furnace, NN),...
7    [(BDZ-160-A, NNP), (Card, NNP), (furnace, NN),...
dtype: object


In [ ]:
print(data_new.head())

0    [(APG, NNP), (STO0045, NNP), (Camping, NNP), (...
1    [(Brand, NNP), (Name, NNP), (STO0045, NNP), (C...
5    [(Model, NNP), (Card, NNP), (furnace, NN), (Wi...
6    [(BDZ-160-A, NNP), (Card, NNP), (furnace, NN),...
7    [(BDZ-160-A, NNP), (Card, NNP), (furnace, NN),...
dtype: object
